In [ ]:
import os
import networkx as nx
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import pickle
import time
import matplotlib.pyplot as plt
import heapq

**FOR MY COLLEAGUES**<br>
Ignore the part with the dataframes, skip to the last two cells (the one with the class definition and the one after that) and run those. Be sure to have downloaded the file I put on drive and put it in a folder called "files". IMO, this is preferable to changing the path in the cell because if we were to do so we would end up uploading versions with different paths at every git push.

In [ ]:
def load_dfs():
    
    if "df_a2q" not in os.listdir("files") \
    or "df_c2q" not in os.listdir("files") \
    or "df_c2a" not in os.listdir("files"):
    
        file_names = ["files/sx-stackoverflow-a2q.txt",
                      "files/sx-stackoverflow-c2q.txt",
                      "files/sx-stackoverflow-c2a.txt"]

        columns = [["user_answering", 
                    "user_questioning", 
                    "time_u"],
                   ["user_commenting", 
                    "user_questioning", 
                    "time_u"],
                   ["user_commenting", 
                    "user_answering", 
                    "time_u"]]

        df_a2q_raw = pd.read_csv(file_names[0],
                                 delim_whitespace=True,
                                 names=columns[0])

        df_c2q_raw = pd.read_csv(file_names[1],
                                 delim_whitespace=True,
                                 names=columns[1])

        df_c2a_raw = pd.read_csv(file_names[2],
                                 delim_whitespace=True,
                                 names=columns[2])
        
        
        dfs_raw = [df_a2q_raw, df_c2q_raw, df_c2a_raw]

        print(df_a2q_raw.shape)

        # add a column for a standard visualization of dates
        for df in dfs_raw:
            df["time_h"] = pd.to_datetime(df["time_u"], unit="s")

        names = ["a2q", "c2q", "c2a"]
        for i, df in enumerate(dfs_raw):
            most_recent = df["time_h"].max()
            name = names[i]
            print(f"Most recent date for dataset {name}: {most_recent}")

        threshold = datetime(year=2014, month=1, day=1, hour=0, minute=0, second=0)
        print(f"Date threshold selected: {threshold}")

        df_a2q = df_a2q_raw[df_a2q_raw["time_h"] > threshold].copy()
        df_c2q = df_c2q_raw[df_c2q_raw["time_h"] > threshold].copy()
        df_c2a = df_c2a_raw[df_c2a_raw["time_h"] > threshold].copy()

        dfs = [df_a2q, df_c2q, df_c2a]
        for df in dfs:
            min_el = df["time_u"].min()
            max_el = df["time_u"].max()
            
            # variation on min max scaler so to have values closer to the max -> to 0 and values
            # closer to the min -> to 1.
            df["weight"] = (max_el - df["time_u"]) / (max_el - min_el)
            
            df.drop("time_u", inplace=True, axis=1)

        with open("files/df_a2q", "wb") as file:
            pickle.dump(df_a2q, file)

        with open("files/df_c2q", "wb") as file:
            pickle.dump(df_c2q, file)

        with open("files/df_c2a", "wb") as file:
            pickle.dump(df_c2a, file)
    
    else:
        
        with open("files/df_a2q", "rb") as file:
            df_a2q = pickle.load(file)
        
        with open("files/df_c2q", "rb") as file:
            df_c2q = pickle.load(file)
            
        with open("files/df_c2a", "rb") as file:
            df_c2a = pickle.load(file)
    
    return df_a2q, df_c2q, df_c2a

In [ ]:
names = ["a2q", "c2q", "c2a"]

In [ ]:
df_a2q, df_c2q, df_c2a = load_dfs()
print(df_a2q.shape)

In [ ]:
df_a2q.head()

In [ ]:
df_c2q.head()

In [ ]:
df_c2a.head()

In [ ]:
dfs = [df_a2q,
       df_c2q,
       df_c2a]

In [ ]:
def time_wrapper(func):
    """
    Decorator to time functions, probably we won't need it in the final version, but for now I'm leaving it 
    in case you need it too. Just put @time_wrapper above any function you want to time and it will print
    the time in seconds it took to run.
    """
    def wrapped(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        print(f"Time elapsed: {time.time()-start}")
        return r
    return wrapped


def visualize_network(graph, edges_to_highlight=[], highlight_nodes=True):
    
    nx_g = nx.MultiGraph(graph)
    black_edges = [edge for edge in nx_g.edges() if edge not in edges_to_highlight]
    nodes_to_highlight=[]
    if highlight_nodes:
        for edge in edges_to_highlight:
            for node in edge:
                if node not in nodes_to_highlight:
                    nodes_to_highlight.append(node)
        blank_nodes = [node for node in nx_g.nodes() if node not in nodes_to_highlight]
    
    plt.figure(figsize=(30, 20))
    pos = nx.spring_layout(nx_g)
    
    nx.draw_networkx_nodes(nx_g, pos, cmap=plt.get_cmap('jet'), node_size = 100, nodelist=nodes_to_highlight, node_color="r")
    nx.draw_networkx_nodes(nx_g, pos, cmap=plt.get_cmap('jet'), node_size = 100, nodelist=blank_nodes, alpha=0.2)
    nx.draw_networkx_edges(nx_g, pos, edgelist=edges_to_highlight, edge_color='r', arrows=True)
    nx.draw_networkx_edges(nx_g, pos, edgelist=black_edges, alpha=0.2, arrows=True)
    plt.show()


class Graph:
    
    def __init__(self):
        self.graph = {}
                
    
    @time_wrapper
    def create_graph(self, sources, names):
        """
        Args:
            sources: dataframes from which to create the graph
            names: a list of string containing the names of the graph used in order to label the edges accordingly
        
        Summary:
            Each node in the graph acts as a key in the dictionary self.graph. Each entry in the dictionary is itself
            another dictionary, indexed by that node's neighbors and pointing to a list with the details of that edge,
            specifically [type_of_interaction, weight, date_human_format].
            So to make things clearer:
            
            self.graph -> Our entire graph, indexed by node
            self.graph[node "A"] -> dictionary indexed by the neighbors of "A"
            self.graph[node "A"][node "B"] -> dictionary indexed by the time the interation took place
            self.graph[node "A"][node "B"][time "x"] -> details (specifically type of interaction and edge weight) of the
                                                        of the interaction between user A and B that took place at time x            
        """
        
        self.graph = {}
    
        for i_source, source in enumerate(sources):
            nodes_column = source.iloc[:, 0]
            nodes = nodes_column.unique()
            type_of_interaction = names[i_source]
            
            for node in tqdm(nodes):
                if node not in self.graph.keys():
                    self.graph[node] = {}
                node_subdf = source[nodes_column == node]
                neighbors = node_subdf.iloc[:, 1].values
                times_h = node_subdf.iloc[:, 2].values
                weights = node_subdf.iloc[:, 3].values
                for i, neighbor in enumerate(neighbors):
                    if neighbor not in self.graph.keys(): # here we want to make sure that if we're working on a subset
                                                          # of data we still obtain a coherent graph. Otherwise we may
                                                          # end up with neighbors that don't exist as actual nodes
                                                          # in our graph.
                        self.graph[neighbor] = {}
                    if neighbor not in self.graph[node].keys():
                        self.graph[node][neighbor] = {}
                        
                    self.graph[node][neighbor][times_h[i]] = {"weight": weights[i], 
                                                              "type_of_interaction": type_of_interaction}
        
    
    @time_wrapper
    def save_graph(self, path):
        """
        Args:
            path: name of file where we want the file stored
        Summary:
            Saves graph as a binary file.
        """
        
        with open(path, "wb") as file:
            pickle.dump(self.graph, file)

    
    @time_wrapper
    def load_graph(self, path):
        """
        Args:
            path: name of file to load the graph from.
        Summary:
            Load graph from binary file.
        """
        
        with open(path, "rb") as file:
            self.graph = pickle.load(file)
    
    
    def get_random_subgraph(self, n_nodes=10000):
        np.random.seed(42)
        bad_starting_node = True
        while bad_starting_node:
            random_node = np.random.choice(list(self.graph.keys()))
            if len(self.graph[random_node].keys()) > 10:
                bad_starting_node = False
        print(random_node)
        print(len(self.graph[random_node].keys()))
        subgraph = {random_node: {}}
        to_check = [random_node]
        i = 0
        
        while len(subgraph.keys()) < n_nodes:
            node = to_check[i]
            if node not in subgraph.keys() or len(subgraph[node].keys()) < 1:
                for neighbor in self.graph[node].keys():
                    if neighbor not in to_check:
                        to_check.append(neighbor)
                    if neighbor not in subgraph.keys():
                        subgraph[neighbor] = {}
                    subgraph[node][neighbor] = self.graph[node][neighbor]
                    if len(subgraph.keys()) >= n_nodes:
                        break
            i += 1
        
        return subgraph
    
    
    def dijkstra(self, start_node, end_node, gr, visualization=True):
        unvisited = []
        for key in gr.keys():
            if key not in unvisited:
                unvisited.append(key)
        visited = []
        
        min_distances = {key: (np.inf, None) for key in gr.keys()}
        min_distances[start_node] = (0, None)
        
        current_node = start_node
        
        while current_node != end_node:
            visited.append(current_node)
            unvisited.remove(current_node)
            for neighbor in gr[current_node].keys():
                weights = [gr[current_node][neighbor][time_key][0] for time_key in gr[current_node][neighbor].keys()]
                weight = np.min(weights)
                tot = min_distances[current_node][0] + weight
                if tot < min_distances[neighbor][0]:
                    min_distances[neighbor] = (tot, current_node)
            
            min_unvisited = np.inf
            found_something = False
            for candidate in unvisited:
                if min_distances[candidate][0] < min_unvisited:
                    min_unvisited = min_distances[candidate][0]
                    current_node = candidate
                    found_something = True
            
            if not found_something:
                break
        
        node_a = None
        node_b = end_node
        
        path = []
        while node_a != start_node:
            node_a = min_distances[node_b][1]
            if node_a == None:
                break
            path.append((node_a, node_b))
            node_b = node_a
        
        if len(path) <= 0:
            print("No path found.")
        
        else:
            path = path[::-1]
            if visualization:
                visualize_network(gr, path)
        
        return path, min_distances
    
    
    @time_wrapper
    def get_neighbors(self, node, interaction="all"):
        """
        Args:
            node: the node we want the neighbors of.
            interaction: the label of the edges we're interested in, default=all, meaning we will obtain all
                         neighbors of that node, regardless of the type of interactions.
        Summary:
            Obtains all the neighbors of a specific node.
        """
        
        if interaction == "all":
            neighbors = list(self.graph[node].keys())
        else:
            neighbors = []
            for neighbor in self.graph[node].keys():
                if self.graph[node][neighbor][0] == interaction:
                    neighbors.append(neighbor)
        
        return neighbors
        

In [ ]:
# Before running this cell, make sure to have the pickle file in a "files" folder, or change the path of the if 
# condition accordingly. This will take some time: on my pc it takes approximately 7 minutes to load the graph.
# The timing decorator will tell you how much it took, so you can plan accordingly.

g = Graph()

if "merged_graph.p" in os.listdir("files"):
    g.load_graph("files/merged_graph.p")
else:
    g.create_graph(dfs, names=names)
    g.save_graph("files/merged_graph.p")

In [ ]:
g = Graph()
g.create_graph(dfs, names)
g.save_graph("files/merged_graph_2.p")

In [ ]:
subg = g.get_random_subgraph(n_nodes=1000)

In [ ]:
np.random.choice(list(subg.keys()), 2)

In [ ]:
#1491895
for key in subg[3043801].keys():
    if len(list(subg[key].keys())) > 3:
        print(key)

In [ ]:
subg[596952].keys()

In [ ]:
g.dijkstra(start_node=3043801,
           end_node=3128099,
           gr=subg)

In [ ]:
visualize_network(subg)

In [ ]:
# This for lightweight graph

g = Graph()
g.load_graph("files/merged_graph_lw.p")

In [ ]:
df_a2q.head()

In [ ]:
visualize_network(g)

## Functionality 4 (Work in progress)

In [ ]:
def get_df_for_time_interval(start, end, dfs, names):
    
    start = datetime.strptime(start, "%d %B %Y %H:%M:%S")
    end = datetime.strptime(end, "%d %B %Y %H:%M:%S")
    print(f"start: {start}")
    print(f"end: {end}")
    
    subdfs = []
    
    for i, df in enumerate(dfs):
        sub_df = df[df["time_h"] >= start]
        sub_df = sub_df[sub_df["time_h"] < end]
        sub_df.columns = ["user_a", "user_b", "time_h", "weight"]
        sub_df["type"] = names[i]
        subdfs.append(sub_df)
    
    return pd.concat(subdfs, axis=0)

In [ ]:
start_1 = "10 June 2015 15:00:00"
end_1 = "10 June 2015 19:00:00"

df_g1 = get_df_for_time_interval(start_1, end_1, dfs, names)

In [ ]:
df_g1.head()

In [ ]:
def get_subgraph_from_df(df):
    subg = {}
    nodes = np.unique(df["user_a"].values)
    for node in nodes:
        
        if node not in subg.keys():
            subg[node] = {}
        
        subdf = df[df["user_a"]==node]
        neighs = subdf["user_b"].values
        times = subdf["time_h"].values
        weights = subdf["weight"].values
        type_of_interaction = subdf["type"].values
        
        for i, neigh in enumerate(neighs):
            if neigh not in subg.keys():
                subg[neigh] = {}
            if neigh not in subg[node].keys():
                subg[node][neigh] = {}
            subg[node][neigh][times[i]] = {"weight": weights[i], 
                                           "type_of_interaction": type_of_interaction[i]}
        
    return subg

In [ ]:
def disconnect_nodes(subg, node_1, node_2):
    
    removed_edges = []
    shortest_path = [node_1]
    
    while len(shortest_path) > 0:
        shortest_path, distances_m = g.dijkstra(start_node=node_1, end_node=node_2, gr=subg, visualization=False)
        cheapest_value = np.inf
        candidate = (None, None)

        while node_b != node_1:
            value, node_a = distances_m[node_b]
            if value < cheapest_value:
                candidate_edge = (key, node_b)
                
        removed_edges.append(candidate_edge)
        subg[candidate_edge[0]].pop(candidate_edge[1])
    
    return removed_edges

In [ ]:
subg_1 = get_subgraph_from_df(df=df_g1)

In [ ]:
visualize_network(subg_1)

In [ ]:
np.random.choice(list(subg_1.keys()), 2)

In [ ]:
disconnect_nodes(subg_1, 2595183, 4930287)

In [ ]:
nx_g = nx.MultiGraph(subg_1)
nx_g.number_of_nodes()

In [ ]:
visualize_network(subg_1)

In [ ]:
df_g1.shape

In [ ]:
len(subg_1.keys())